# Hello "Compose"

__Aplicações com multiplos containers__

Em uma aplicação simples, de apenas 1 container fica claro as vantagens que a sua utilização trás, como agilidade, integridade, isolamento, etc... Porém para uma aplicação maior, envolvendo diversos containers, o gerenciamento de todos pode ser tornar trabalhoso e até mesmo confuso, e o a principal desvantagem seria a perda das caracteristicas que tornam o uso de containers tão interessantes.

Agora suponha que em um único arquivo, do tipo __.yaml__ com declarações do tipo __service__ e a identação correta, será possível declarar os containers, fazer as conexões necessárias e apartir de um único comando a pilha de declarações é fornecida. Problema resolvio então, prazer __Docker Compose__

__Caracteristicas:__

* Ferramenta externa (open source)
    * Escrita em Python (Com sua _engine_ em GO)
    
    
* Conjunto de container apartir de um único arquivo
    * Containers são serviços - __microsserviços__ de uma aplicação
    * Único comando --> cria e inicia todos os serviços
    
    
* Multíplos _ambiente de desenvolvimento_ em um único host
    * É possível ter separações por diretórios
    
    
* Documentação e configuração de aplicações
    * Com o uso de docker compose praticamente teremos uma documentação em código!

# Sem o arquivo Compose, Nada feito!

O arquivo _compose:_


* Supora os formatos __.yml__ (ou __.yaml__ ) e __.json__

## Declaração:

Pode ser feito em qualquer editor

In [ ]:
#Primeiro o cabeçalho, declarando a versão do compose utilizada
version:"3"

#agora o services, que neste contexto os containers são vistos como services
#dentro de cada service as suas config options, ou seja, as declarações de cada container
services:
    app:
        build:./dir
        image:webapp:tag
        ports:
            -"6379"
            networks:
                -frontend* #Footer
    
    db:
        image:postgres:9.4
        volumes:
            #Footer
            -db-data:/var/lib/postgresq/data
        networks:
            -backend #Footer
            
    #Footer declara as config personalizadas informadas anteriormente
    networks:
        frontend
        backend
    volumes:
        dbdata

## Config Options

__Declaram as opções de cada container__

* __build__ - contrói uma imagem a partir do arquivo Dockerfile informado;
* __image__ - nada mais é doque o __docker pull__ informa nome e tag da imagem. se atrelado a config _build_ contruirá uma imagem com nome e tag informados
* __environment__ - adiciona variáveis de ambientes
* __ports__ - expõe as portas do service ( _expose_ ). Padrão " _HOST:CONTAINER_ "
* __networks__ - informa as docker networks que o container fará parte. Padrão: network criada com o nome do projeto
* __volumes__ - oferece o recurso de _bind mount_ e _named volumes (docker volumes)_

# Nova CLI

Apesar de nova sua estrutura é muito semelhante a do _docker run_

__docker-compose OPTIONS COMMAND ARGS__


* __OPTIONS:__ --file, -f, -p, --project-name, ...
    * Por padrão o nome do projeto é o diretório atual
* __COMMAND ARGS:__ ps, up -d, down, ...

__Principais comandos__

* __docker-compose up OPTIONS SERVICE_NAME:__
    * Contróia toda a stack como builds, (re)creates, attaches, etc...
    * Tudo deve ser feito dentro do diretório em que se encontra o arquivo do docker compose
        * __OPTIONS:__ -d, --force-recreate
            * __id__ - Rodar em background
            * __--force-recreate__ - O docker compose utiliza-se do _cache_ , então caso o container não tenha sido modificado ele não será recriado, então este comando força com que ele seja recriado! OBS: se alguma modificação foi feita e este comando não foi utilizado, obviamente o container será recriado
        * __Service_Name:__ 
            * Serviço (container) da stack de aplicação
            
            
* __docker-compose ps OPTIONS:__
    * Lista todos os services (containers)
    * __OPTIONS:__
        * __-q__ - exibi somente os IDs dos containers
        
        
* __docker-compose logs OPTIONS SERVICE_NAME:__
    * Mostra os logs de saida dos serviços
    * __OPTIONS:__
        * __-f__ - Insere o conceito de leitura dinamica (Não precisa do Ctrl+C para sair)
        
        
* __docker-compose images OPTIONS SERVICE:__
    * Lista as imagens usadas pelos containers criados
    * __OPTIONS:__
        * __-q__ - Exibi somente as IDs dos containers


# Exemplo

Para um provisionamento de um Repositório Online de Arquivos usando o Owncloud, com conexão a uma base de dados MySQL, os comandos Docker podem ser usados.

A fim de obter melhor entendimento sobre a manipulação do arquivo de configuração do compose, essa tarefa requer que você, a partir dos comandos docker abaixo, transcreva-os para um arquivo yml e crie um projeto Docker Compose:

In [ ]:
docker network create net-backend 
docker run --name mysql --restart=always --network=net-backend 
-v /home/data:/var/lib/mysql 
-v /etc/localtime:/etc/localtime:ro 
-e MYSQL_ROOT_PASSWORD=pass -d mysql:5.5 

docker run --name owncloud --restart=always --network=net-backend 
-p 9090:80 -v /home/apps:/var/www/html/apps 
-v /home/config:/var/www/html/config 
-v /home/data:/var/www/html/data 
-v /etc/localtime:/etc/localtime:ro -d owncloud

__Arquivo Docker-Compose.yml:__

In [ ]:
version: '3'

services:
  mysql:
    build:./home/data
    image:mysql:5.5
    volumes:
      -/home/data:/var/lib/mysql
      -/etc/localtime:/etc/localtime:ro
    environment:
      -MYSQL_ROOT_PASSWORD=pass
    ports:
      networks:
        -net-backend

  owncloud:
    build:./home/apps
    image:owncloud
    volumes:
      -/home/data:/var/www/html/apps
      -/home/config:/var/www/html/config
      -/home/data:/var/www/html/data
      -/etc/localtime:/etc/localtime:ro
    ports:
      -"9090"
        networks:
          -net-backend

  networks:
    backend

  volumes:
    /home/data
    /etc/localtime
    /home/config